<a href="https://colab.research.google.com/github/Bio2Byte/public_notebooks/blob/main/ACPype_API_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACPYPE API Reference

[ACPYPE](https://bio2byte.be/acpype/) provides a user-friendly HTTP API that allows users to submit molecules in a variety of formats for processing using the ACPYPE software.

Our API enables users to specify a wide range of options for processing, including charge method, net charge, and atom type.

After submitting their molecule, users can query the result files generated by ACPYPE in the background using a simple hash ID.

![https://bio2byte.be/static_tools/images/workflow_acpype.jpg](https://bio2byte.be/static_tools/images/workflow_acpype.jpg)

## About ACPYPE

The ACPYPE Portal was designed to generate topology parameters files for unusual organic chemical compounds. It is based on ANTECHAMBER and so far, ACPYPE is able to work for CNS/XPLOR, GROMACS, CHARMM and AMBER.

The main scope for ACPYPE Server is to help to pave the way for automatic molecular dynamics simulations involving molecules with unknown parameters, like for example, in complexes of protein and inhibitor, where the ligand is usually an unusual chemical compound.

ACPYPE stands for AnteChamber PYthon Parser interfacE and is pronounced "ace + pipe".

### What the ACPYPE Server does
It will take either a SMILES input or files in format PDB, MDL or MOL2 of a small organic molecule without open valence and assign charges and force ﬁeld parameterizations according to GAFF (Generalised Amber Force Field). There are some options that can be applied to a submitting project.
For charge, there are three methods:

bcc: for semi-epirical AM1-BCC, parameterized to reproduce HF/6- 31G* RESP charges; slow, but with good cost/benefit (default)
Gasteiger method, very fast but less accurate
user: for a MOL2 file with charges already calculated, via, e.g., R.E.D.-III
For net charge, set a integer value for the project, or let ACPYPE guess the net charge for the molecule.
For atom type, set GAFF (default), GAFF2 or AMBER. If set AMBER, ACPYPE/antechamber will try to set parameters and atom types according to AMBER14SB forcefield. Case it fails, GAFF parameters (but with AMBER atom types) will be used.

### What the ACPYPE Server doesn't do
It will not work with organic molecule with open valences; containing others atoms than C, N, O, S, P, H, F, Cl, Br and I; or covalently bonded to another molecule. If one wants parameters for a modified amino acid residue, one way of getting it is by neutralising the N- and C- termini and then fit manually the additional parameters to the modified residue.

### Citations
If you use this resource, please cite:

> SOUSA DA SILVA, A. W. & VRANKEN, W. F. ACPYPE - AnteChamber PYthon Parser interfacE. BMC Research Notes 2012, 5:367
KAGAMI, L. P., SOUSA DA SILVA, A. W., DÍAZ, A., & VRANKEN, W. F. The ACPYPE web server for small molecule MD topology generation. Manuscript submitted.
If you use non-uniform 1-4 scale factor conversion (e.g. if using GLYCAM06), please cite:

> BERNARDI, A., FALLER, R., REITH, D., and KIRSCHNER, K. N. ACPYPE update for nonuniform 1-4 scale factors: Conversion of the GLYCAM06 force field from AMBER to GROMACS. SoftwareX 10 (2019), 100241.

### Disclaimer
This service is provided with ABSOLUTELY NO WARRANTY and holds no liabilities. If you decide to use it, bear in mind that your data and potential results are not stored with encryption, and the service administration has access to it. Furthermore, the data and results will be eventually removed after two weeks from the time of submission, whether your job has finished or not.

## Submitting a ligand
Depending on your input, please execute one of the two available scenarios.

In [ ]:
# Import the necessary libraries
import json
import requests

In [ ]:
location = None
hash_id = None

### Scenario A: Using SMILES representation

In [ ]:
url = "https://bio2byte.be/acs/api"
print("POST", url)

# Define request payload
payload = {
    "inputFile": None,
    "file_name": "OXYGEN_FROM_API",
    "token": "RTV5GZHc5M",
    "charge_method": "bbc",
    "net_charge": None,
    "atom_type": "gaff",
    "email": None,
    "smiles": "O"
}

# Send POST request
response = requests.post(url, json=payload)

# Process the response
if response.status_code > 200 and response.status_code < 300:
    data = response.json()
    location = data["Location"]
    hash_id = data["hash_id"]
    message = data["message"]
    print("Prediction job submitted successfully!")
    print(f"Location: {location}")
    print(f"hash_id: {hash_id}")
    print(f"Message: {message}")
else:
    print("Failed to submit prediction job. Status Code:", response.status_code)

### Scenario B: Using a file with coordinates
Please upload to Google Colab your input file (PDB, MDL or MOL2 of a small organic molecule)

In [ ]:
url = "https://bio2byte.be/acs/api"
print("POST", url)

with open("/content/EXAMPLE.pdb", "rb") as input_file_handler:
    file_content = input_file_handler.read().decode("utf-8")

# Define request payload
payload = {
    "inputFile": file_content,
    "file_name": "EXAMPLE.pdb",
    "token": "RTV5GZHc5M",
    "charge_method": "bbc",
    "net_charge": None,
    "atom_type": "gaff",
    "email": None,
    "smiles": None
}

# Send POST request
response = requests.post(url, json=payload)

# Process the response
if response.status_code > 200 and response.status_code < 300:
    data = response.json()
    location = data["Location"]
    hash_id = data["hash_id"]
    message = data["message"]
    print("Prediction job submitted successfully!")
    print(f"Location: {location}")
    print(f"hash_id: {hash_id}")
    print(f"Message: {message}")
else:
    print("Failed to submit prediction job. Status Code:", response.status_code)

## Querying the submission status via Hash ID

In [ ]:
url = f"https://bio2byte.be/acs{location}"
print("GET", url)

# Make the API request and store the JSON response in a variable
response = requests.get(url)

# Check the response status code
if response.status_code >= 400:
    print("Failed to fetch prediction job. Response status Code:", response.status_code)
elif response.status_code >= 202:
    print("Still processing your request. Please try again in a minute. Response status code: ", response.status_code)

    # Extract the JSON response
    json_response = response.json()
    print(json_response)
else:
    print("Response status code:", response.status_code)
    # Extract the JSON response
    json_response = response.json()

    id = json_response['id']
    creation_date = json_response['creation_date']
    token = json_response['token']
    hash_id = json_response['hash_id']
    result_request = json_response['result_request']
    location = result_request['location']
    status = result_request['status']

    print("Response fields:")
    print(f"id: {id}")
    print(f"creation_date: {creation_date}")
    print(f"token: {token}")
    print(f"hash_id: {hash_id}")
    print(f"location: {location}")
    print(f"status: {status}")
    
    print("You job files are available!")

## Fetching the submission results via Hash ID

In [ ]:
# Make the API request and store the JSON response in a variable
url = f"https://bio2byte.be/acs/api/{hash_id}/"
print("GET", url)

response = requests.get(url)

# Check the response status code
if response.status_code >= 400:
    # Display an error message if the request fails
    print("Failed to fetch prediction job. Status Code:", response.status_code)
elif response.status_code >= 300:
    print("Still working on the prediction job, please try again in a minute. Status Code:", response.status_code)
else:
    # Extract the JSON response
    json_response = response.json()

    # Loop through each result and save it to a separate file
    for i, result in enumerate(json_response["results"], start=1):
        for key, value in result.items():
            
            # Save each result to a file
            print(f"Saving {key} content to result_{i}_{key}.txt")
            
            with open(f"result_{i}_{key}.txt", "w") as f:
                f.write(value)
    
    print("Files saved with success")

In [ ]:
# Compress the result files
!zip -r acpype_results.zip /content/*.txt

# Download results
from google.colab import files
files.download("/content/acpype_results.zip")